In [1]:
!python -m pip install paho-mqtt

In [2]:
# IMPORT DEPENDENCIES------------------------------------------------------------------------------
import json
import os
import logging
from datetime import datetime
from re import I
import sys
import paramiko
from PIL import Image 
import time
import numpy 
import cv2
import threading
import UI_cli
import csv 

from opentrons import opentronsClient

from biologic import connect, BANDWIDTH, I_RANGE, E_RANGE
from biologic.techniques.ocv import OCVTechnique, OCVParams, OCVData
from biologic.techniques.peis import PEISTechnique, PEISParams, SweepMode, PEISData
from biologic.techniques.ca import CATechnique, CAParams, CAStep, CAData
from biologic.techniques.cpp import CPPTechnique, CPPParams, CPPData
from biologic.techniques.pzir import PZIRTechnique, PZIRParams, PZIRData
from biologic.techniques.cv import CVTechnique, CVParams, CVStep, CVData
from biologic.techniques.lp import LPTechnique, LPParams, LPStep, LPData
from biologic.techniques.cp import CPTechnique, CPParams, CPStep, CPData

import pandas as pd

# HELPER FUNCTIONS---------------------------------------------------------------------------------

# define helper functions to manage solution
def fillWell(
    opentronsClient,
    strLabwareName_from,
    strWellName_from,
    strOffsetStart_from,
    strPipetteName,
    strLabwareName_to,
    strWellName_to,
    strOffsetStart_to,
    intVolume: int,
    fltOffsetX_from: float = 0,
    fltOffsetY_from: float = 0,
    fltOffsetZ_from: float = 0,
    fltOffsetX_to: float = 0,
    fltOffsetY_to: float = 0,
    fltOffsetZ_to: float = 0,
    intMoveSpeed : int = 100, 
    needMixing: bool = False,
) -> None:
    '''
    function to manage solution in a well because the maximum volume the opentrons can move is 1000 uL
    
    Parameters
    ----------
    opentronsClient : opentronsClient
        instance of the opentronsClient class

    strLabwareName_from : str
        name of the labware to aspirate from

    strWellName_from : str
        name of the well to aspirate from

    strOffset_from : str
        offset to aspirate from
        options: 'bottom', 'center', 'top'

    strPipetteName : str
        name of the pipette to use

    strLabwareName_to : str
        name of the labware to dispense to

    strWellName_to : str
        name of the well to dispense to

    strOffset_to : str
        offset to dispense to
        options: 'bottom', 'center', 'top'  

    intVolume : int
        volume to transfer in uL    

    intMoveSpeed : int
        speed to move in mm/s
        default: 100
    '''
    
    # while the volume is greater than 1000 uL
    while intVolume > 1000:
        # move to the well to aspirate from
        opentronsClient.moveToWell(strLabwareName = strLabwareName_from,
                                   strWellName = strWellName_from,
                                   strPipetteName = strPipetteName,
                                   strOffsetStart = 'top',
                                   fltOffsetX = fltOffsetX_from,
                                   fltOffsetY = fltOffsetY_from,
                                   intSpeed = intMoveSpeed)
                                   
        time.sleep(0.01)
        
        # aspirate 1000 uL
        opentronsClient.aspirate(strLabwareName = strLabwareName_from,
                                 strWellName = strWellName_from,
                                 strPipetteName = strPipetteName,
                                 intVolume = 1000,
                                 strOffsetStart = strOffsetStart_from,
                                 fltOffsetX = fltOffsetX_from,
                                 fltOffsetY = fltOffsetY_from,
                                 fltOffsetZ = fltOffsetZ_from)

        time.sleep(0.01)
        
        # move to the well to dispense to
        opentronsClient.moveToWell(strLabwareName = strLabwareName_to,
                                   strWellName = strWellName_to,
                                   strPipetteName = strPipetteName,
                                   strOffsetStart = 'top',
                                   fltOffsetX = fltOffsetX_to,
                                   fltOffsetY = fltOffsetY_to,
                                   intSpeed = intMoveSpeed)

        time.sleep(0.01)
        
        # dispense 1000 uL
        opentronsClient.dispense(strLabwareName = strLabwareName_to,
                                 strWellName = strWellName_to,
                                 strPipetteName = strPipetteName,
                                 intVolume = 1000,
                                 strOffsetStart = strOffsetStart_to,
                                 fltOffsetX = fltOffsetX_to,
                                 fltOffsetY = fltOffsetY_to,
                                 fltOffsetZ = fltOffsetZ_to)

        time.sleep(0.01)
        
        opentronsClient.blowout(strLabwareName = strLabwareName_to,
                                strWellName = strWellName_to,
                                strPipetteName = strPipetteName,
                                strOffsetStart = strOffsetStart_to,
                                fltOffsetX = fltOffsetX_to,
                                fltOffsetY = fltOffsetY_to,
                                fltOffsetZ = fltOffsetZ_to)

        time.sleep(0.01)
        
        # subtract 1000 uL from the volume
        intVolume -= 1000
    
        # move to the well to aspirate from
    opentronsClient.moveToWell(strLabwareName = strLabwareName_from,
                               strWellName = strWellName_from,
                               strPipetteName = strPipetteName,
                               strOffsetStart = 'top',
                               fltOffsetX = fltOffsetX_from,
                               fltOffsetY = fltOffsetY_from,
                               intSpeed = intMoveSpeed)
    
    # aspirate the remaining volume
    opentronsClient.aspirate(strLabwareName = strLabwareName_from,
                             strWellName = strWellName_from,
                             strPipetteName = strPipetteName,
                             intVolume = intVolume,
                             strOffsetStart = strOffsetStart_from,
                             fltOffsetX = fltOffsetX_from,
                             fltOffsetY = fltOffsetY_from,
                             fltOffsetZ = fltOffsetZ_from)
    
    # move to the well to dispense to
    opentronsClient.moveToWell(strLabwareName = strLabwareName_to,
                               strWellName = strWellName_to,
                               strPipetteName = strPipetteName,
                               strOffsetStart = 'top',
                               fltOffsetX = fltOffsetX_to,
                               fltOffsetY = fltOffsetY_to,
                               intSpeed = intMoveSpeed)
    
    # dispense the remaining volume
    opentronsClient.dispense(strLabwareName = strLabwareName_to,
                             strWellName = strWellName_to,
                             strPipetteName = strPipetteName,
                             intVolume = intVolume,
                             strOffsetStart = strOffsetStart_to,
                             fltOffsetX = fltOffsetX_to,
                             fltOffsetY = fltOffsetY_to,
                             fltOffsetZ = fltOffsetZ_to)
    
    # blowout
    opentronsClient.blowout(strLabwareName = strLabwareName_to,
                            strWellName = strWellName_to,
                            strPipetteName = strPipetteName,
                            strOffsetStart = strOffsetStart_to,
                            fltOffsetX = fltOffsetX_to,
                            fltOffsetY = fltOffsetY_to,
                            fltOffsetZ = fltOffsetZ_to)
    
    if needMixing: 
        for i in range(6):
            print("mixing cycle: ", i+1)
            opentronsClient.aspirate(strLabwareName = strLabwareName_to,
                                strWellName = strWellName_to,
                                strPipetteName = strPipetteName,
                                intVolume = 1000,
                                strOffsetStart = strOffsetStart_to,
                                fltOffsetX = fltOffsetX_to,
                                fltOffsetY = fltOffsetY_to,
                                fltOffsetZ = -30)
                            
            time.sleep(0.01)

            opentronsClient.dispense(strLabwareName = strLabwareName_to,
                                strWellName = strWellName_to,
                                strPipetteName = strPipetteName,
                                intVolume = 1000,
                                strOffsetStart = strOffsetStart_to,
                                fltOffsetX = fltOffsetX_to,
                                fltOffsetY = fltOffsetY_to,
                                fltOffsetZ = -30)
        
    return


# define helper function to wash reactor
def washReactor(oc,
                strID_NISreactor,
                strWell2Test,
                prePictureName, 
                postPictureName,
                pumps):
    '''
    function to wash reactor

    Parameters
    ----------
    opentronsClient : opentronsClient
        instance of the opentronsClient class

    strLabwareName : str
        name of the labware to wash electrode in

    intCycle : int
        number of cycles to wash electrode

    '''

    # rinse cycle 4 times: nozzle immerse 3 times
    # pick up nozzle 
    oc.moveToWell(
            strLabwareName=strID_electrodeTipRack,
            strWellName='B1',
            strPipetteName="p1000_single_gen2",
            strOffsetStart='top',
            fltOffsetX=0.5,
            fltOffsetY=0.5,
            fltOffsetZ=2,
            intSpeed=50
        )

    time.sleep(0.01)

    oc.pickUpTip(
            strLabwareName=strID_electrodeTipRack,
            strWellName='B1',
            strPipetteName="p1000_single_gen2",
            strOffsetStart='top',
            fltOffsetX=0.5,
            fltOffsetY=0.5
        )

    time.sleep(0.01)

    oc.moveToWell(
            strLabwareName=strID_NISreactor,
            strWellName=strWell2Test,
            strPipetteName='p1000_single_gen2',
            strOffsetStart='top',
            fltOffsetX=0.3,
            fltOffsetY=0.5,
            fltOffsetZ=-30,
            intSpeed=50
        )

    time.sleep(0.01)

    pumps.on(3, 10000)  # out for 10 s (auto-off)
    time.sleep(11)

    oc.moveToWell(
            strLabwareName=strID_NISreactor,
            strWellName=strWell2Test,
            strPipetteName='p1000_single_gen2',
            strOffsetStart='top',
            fltOffsetX=0.3,
            fltOffsetY=0.5,
            fltOffsetZ=-40,
            intSpeed=50
        )

    pumps.on(3, 10000)  # out for 10 s (auto-off)
    time.sleep(11)

    oc.moveToWell(
            strLabwareName=strID_NISreactor,
            strWellName=strWell2Test,
            strPipetteName='p1000_single_gen2', 
            strOffsetStart='top',
            fltOffsetX=0.3,
            fltOffsetY=0.5,
            fltOffsetZ=-54,
            intSpeed=50
        )

    time.sleep(0.01)

    pumps.on(3, 10000)  # out for 10 s (auto-off)
    time.sleep(11)

    oc.moveToWell(
            strLabwareName=strID_electrodeTipRack,
            strWellName='B1',
            strPipetteName="p1000_single_gen2",
            strOffsetStart='top',
            fltOffsetX=0.5,
            fltOffsetY=0.5,
            fltOffsetZ=2,
            intSpeed=50
        )

    # put nozzle back to tip rack
    oc.moveToWell(
            strLabwareName=strID_electrodeTipRack,
            strWellName='B1',
            strPipetteName="p1000_single_gen2",
            strOffsetStart='top',
            fltOffsetX=0.5,
            fltOffsetY=0.5,
            fltOffsetZ=2,
            intSpeed=50
        )

    time.sleep(0.01)

    oc.dropTip(
            strLabwareName=strID_electrodeTipRack,
            boolDropInDisposal=False,
            strWellName='B1',
            strPipetteName="p1000_single_gen2",
            strOffsetStart='top',
            fltOffsetX=0.5,
            fltOffsetY=0.5,
            fltOffsetZ=-88
        )
    
    time.sleep(0.01)
    
    oc.moveToWell(
        strLabwareName=strID_electrodeTipRack,
        strWellName='B1',
        strPipetteName="p1000_single_gen2",
        strOffsetStart='top',
        fltOffsetX=0.5,
        fltOffsetY=0.5,
        fltOffsetZ=20,
        intSpeed=50
    )

    time.sleep(0.01)

    take_picture(oc, strID_NISreactor, strWell2Test, prePictureName, well_path)
    logging.info("Taken pre-wash picture.")

    time.sleep(0.01)

    # pick up nozzle 
    oc.moveToWell(
            strLabwareName=strID_electrodeTipRack,
            strWellName='B1',
            strPipetteName="p1000_single_gen2",
            strOffsetStart='top',
            fltOffsetX=0.5,
            fltOffsetY=0.5,
            fltOffsetZ=2,
            intSpeed=50
        )

    time.sleep(0.01)

    oc.pickUpTip(
            strLabwareName=strID_electrodeTipRack,
            strWellName='B1',
            strPipetteName="p1000_single_gen2",
            strOffsetStart='top',
            fltOffsetX=0.5,
            fltOffsetY=0.5
        )

    time.sleep(0.01)

    oc.moveToWell(
            strLabwareName=strID_NISreactor,
            strWellName=strWell2Test,
            strPipetteName='p1000_single_gen2', 
            strOffsetStart='top',
            fltOffsetX=0.3,
            fltOffsetY=0.5,
            fltOffsetZ=-54,
            intSpeed=50
        )

    time.sleep(0.01)

    for i in range(4):
        pumps.on(2, 2000)  # add H2O for 2 s (auto-off)
        time.sleep(3)
        pumps.on(3, 10000)  # out for 10 s (auto-off)
        time.sleep(11)

    # put nozzle back to tip rack
    oc.moveToWell(
            strLabwareName=strID_electrodeTipRack,
            strWellName='B1',
            strPipetteName="p1000_single_gen2",
            strOffsetStart='top',
            fltOffsetX=0.5,
            fltOffsetY=0.5,
            fltOffsetZ=2,
            intSpeed=50
        )

    time.sleep(0.01)

    oc.dropTip(
            strLabwareName=strID_electrodeTipRack,
            boolDropInDisposal=False,
            strWellName='B1',
            strPipetteName="p1000_single_gen2",
            strOffsetStart='top',
            fltOffsetX=0.5,
            fltOffsetY=0.5,
            fltOffsetZ=-88
        )
    
    time.sleep(0.01)

    oc.moveToWell(
        strLabwareName=strID_electrodeTipRack,
        strWellName='B1',
        strPipetteName="p1000_single_gen2",
        strOffsetStart='top',
        fltOffsetX=0.5,
        fltOffsetY=0.5,
        fltOffsetZ=20,
        intSpeed=50
    )

    time.sleep(0.01)

    take_picture(oc, strID_NISreactor, strWell2Test, postPictureName, well_path)
    logging.info("Taken post-wash picture.")

    logging.info("Finished washing reactor in well %s.", strWell2Test)
    return


def take_picture(oc, strID_NISreactor, strWellName, imageName, well_dir): 
    oc.moveToWell(strLabwareName = strID_NISreactor,
                strWellName = strWellName,
                strPipetteName = 'p1000_single_gen2',
                strOffsetStart = 'top',
                fltOffsetX = 19.0,
                fltOffsetY = 77.0,
                fltOffsetZ = 50,
                intSpeed = 50)
    time.sleep(3)  # wait for 2 seconds to stabilize
    oc.lights(False)

    # === CONFIGURATION ===
    hostname = '192.168.0.108'

    # hostname = '100.66.74.87'  # ⬅️ Replace this with your Raspberry Pi's real IP address
    username = 'ot2-pi'
    password = '1144'
    remote_image_path = '/home/ot2-pi/remote_image.jpg'
    local_image_path = 'remote_image.jpg'

    # === CONNECT TO PI OVER SSH ===
    print(f"[+] Connecting to {hostname}...")
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(hostname, username=username, password=password)
    print("[+] SSH connection established.")

    # === RUN REMOTE CAMERA SCRIPT ===
    command = (
        "python3 -c \""
        "from picamera2 import Picamera2; import time; "
        "picam2 = Picamera2(); "
        "config=picam2.create_still_configuration(main={'size': (2028, 1520)}); "
        "picam2.configure(config); "
        "picam2.set_controls({'AwbEnable': True}); "
        "picam2.start(); "
        "time.sleep(2); "
        f"picam2.capture_file('{remote_image_path}'); "
        "picam2.close(); "
        "print('Image captured');"
        "\""
    )

    print("[+] Capturing image on the Pi...")
    stdin, stdout, stderr = ssh.exec_command(command)
    stdout_output = stdout.read().decode()
    stderr_output = stderr.read().decode()

    local_image_path = os.path.join(well_dir, imageName)

    if stderr_output:
        print("[-] Error during image capture:")
        print(stderr_output)
    else:
        print("[+] Remote output:")
        print(stdout_output)

    # === DOWNLOAD IMAGE FROM PI ===
    print("[+] Downloading image to laptop...")
    sftp = ssh.open_sftp()
    sftp.get(remote_image_path, local_image_path)
    sftp.close()
    ssh.close()
    print(f"[+] Image downloaded to {local_image_path}")
    oc.lights(True)


def getPipetteTipLocById(intId) -> str:
    if intId > 96 or intId < 1:
        raise Exception("Pipette id out of range.")
    return chr(ord('A') + ((intId - 1) // 12)) + str((intId - 1) % 12 + 1)

def allocate_from_sources(sources_by_plate: dict, solution_name:str, required_uL: int):
    """
    Search through 'sources_by_plate', which is a dict of dicts mapping well names to remaining uL,
    mutates sources_by_plate[*][*]['remaining_uL']
    returns plan = [(plate_id, well_name, uL_to_take), ...] 
    """
    plan = []
    need = required_uL
    
    for plate_id, wells in sources_by_plate.items():
        for well_name, info in wells.items():
            if info.get("solution") != solution_name:
                continue

            remain = info.get("remaining_uL", 0)
            if remain <= 0 or need <= 0:
                continue

            take = min(remain, need)
            info['remaining_uL'] = remain - take  # mutate remaining amount
            plan.append( (plate_id, well_name, take) )
            need -= take

            if need <= 0:
                break
        if need <= 0:
                break
        
    if need > 0:
        raise Exception(f"Not enough {solution_name} available to allocate {required_uL} uL.")

    return plan 

def fillWell_autoSource(
    opentronsClient,
    sources_by_plate: dict,
    solution_name: str,
    strPipetteName: str,
    strLabwareName_to: str,
    strWellName_to: str,
    strOffsetStart_from: str = "bottom",
    strOffsetStart_to: str = "bottom",
    totalVolume_uL: int = 0,
    fltOffsetX_from: float = 0,
    fltOffsetY_from: float = 0,
    fltOffsetZ_from: float = 0,
    fltOffsetX_to: float = 0,
    fltOffsetY_to: float = 0,
    fltOffsetZ_to: float = 0,
    intMoveSpeed: int = 100,
    needMixing: bool = False,
    experimentName: str = "",
):
    """
    Auto-pick source wells and reduce their remaining amounts while transferring 'totalVolume_uL'
    into (strLabwareName_to, strWellName_to) using your existing fillWell().
    """
    plan = allocate_from_sources(sources_by_plate, solution_name, totalVolume_uL)  # mutates sources_by_plate
    record_experiment_data(strMetadataPath, experimentName, "solutionAdded", plan)
    for plate_id, well_from, vol_uL in plan:
        fillWell(
            opentronsClient=opentronsClient,
            strLabwareName_from=plate_id,
            strWellName_from=well_from,
            strOffsetStart_from=strOffsetStart_from,
            strPipetteName=strPipetteName,
            strLabwareName_to=strLabwareName_to,
            strWellName_to=strWellName_to,
            strOffsetStart_to=strOffsetStart_to,
            intVolume=vol_uL,
            fltOffsetX_from=fltOffsetX_from,
            fltOffsetY_from=fltOffsetY_from,
            fltOffsetZ_from=fltOffsetZ_from,
            fltOffsetX_to=fltOffsetX_to,
            fltOffsetY_to=fltOffsetY_to,
            fltOffsetZ_to=fltOffsetZ_to,
            intMoveSpeed=intMoveSpeed,
            needMixing=needMixing,
        )

def getWellName(index: int) -> str:
    if not (1 <= index <= 15):
        raise ValueError("Index out of range (1-15)")

    rows = ["A", "B", "C"]
    num_rows = 3

    # column-major ordering:
    row_index = (index - 1) % num_rows
    col_index = (index - 1) // num_rows + 1

    return f"{rows[row_index]}{col_index}"

def wellNameToIndex(wellName: str) -> int:
    wellName = wellName.strip().upper()
    row = ord(wellName[0]) - ord('A')
    col = int(wellName[1:]) - 1
    return row * 5 + col + 1

class VideoRecorder:
    def __init__(self, camera_index=0, width=1280, height=720, fps=30, out_path="experiment.mp4"):
        self.camera_index = camera_index
        self.width = width
        self.height = height
        self.fps = fps
        self.out_path = out_path

        self.cap = None
        self.out = None
        self.thread = None
        self.running = False

    def start(self):
        if self.running:
            print("Camera is already running.")
            return

        self.cap = cv2.VideoCapture(self.camera_index, cv2.CAP_DSHOW)
        if not self.cap.isOpened():
            raise RuntimeError("Could not open camera")

        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, self.width)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.height)
        self.cap.set(cv2.CAP_PROP_FPS, self.fps)

        # Define the codec and create VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        self.out = cv2.VideoWriter(self.out_path, fourcc, self.fps, (self.width, self.height))
        self.running = True
        self.thread = threading.Thread(target=self._record_loop, daemon=True)
        self.thread.start()

        logging.info(f"Video recording started -> {self.out_path}")

    def _record_loop(self):
        while self.running:
            ok, frame = self.cap.read()
            if not ok:
                logging.error("Failed to read frame from camera")
                continue
            self.out.write(frame)
            time.sleep(1 / self.fps)

    def stop(self):
        """Stop the video recording."""
        if not self.running:
            print("Camera is not running.")
            return
        self.running = False
        self.thread.join()
        self.cap.release()
        self.out.release()
        cv2.destroyAllWindows()
        logging.info(f"Video recording stopped -> {self.out_path}")

VALID_WELLS = {f"{row}{col}" for row in ["A", "B", "C"] for col in range(1, 6)}

def load_experiment_csv(path):
    wells = []
    errors = []
    seen_wells = set()

    with open(path, newline="") as f:
        reader = csv.DictReader(f)
        line_num = 1

        for row in reader:
            line_num += 1

            raw_name = (row.get("wellName") or "").strip()
            if not raw_name:
                continue
            well_name = raw_name.upper()

            if well_name not in VALID_WELLS:
                errors.append(f"Line {line_num}: Invalid well name '{well_name}'. Allowed: A1-C5.")
                continue

            if well_name in seen_wells:
                errors.append(f"Line {line_num}: Duplicate well name '{well_name}'.")
                continue
            seen_wells.add(well_name)

            try: 
                well_id = int(row["well ID"])
            except: 
                well_id = None
                errors.append(f"Line {line_num}: Invalid well ID '{row.get('well ID')}'.")

            def parse_float(field, name):
                val_str = (row.get(field) or "").strip()
                if not val_str:
                    return None
                try:
                    return float(val_str)
                except:
                    errors.append(f"Line {line_num}: Invalid float value '{val_str}' in '{name}'.")
                    return None

            temperature_C = parse_float("temperature_C", "temperature_C")
            depositionCurrent_mA = parse_float("depositionCurrent_mA", "depositionCurrent_mA")
            depositionTime_s = parse_float("depositionTime_s", "depositionTime_s")
            
            well = {
                "well_id": well_id,
                "well_name": well_name,
                "temperature_C": temperature_C,
                "depositionCurrent_mA": depositionCurrent_mA,
                "depositionTime_s": depositionTime_s,
                "solutions": []
            }

            for label in ["A", "B", "C", "D"]:
                name_key = f"solution {label} name"
                volume_key = f"solution {label} volume_mL"

                name = (row.get(name_key) or "").strip()
                vol_raw = (row.get(volume_key) or "").strip()

                if not name and not vol_raw:
                    continue  # skip empty solution

                if not name: 
                    errors.append(f"Line {line_num}: Missing name for solution {label}.")
                    continue

                if not vol_raw:
                    errors.append(f"Line {line_num}: Missing volume for solution {label}.")
                    continue

                try: 
                    volume_mL = float(vol_raw)
                    if volume_mL <= 0:
                        raise ValueError("Volume must be positive.")
                except:
                    errors.append(f"Line {line_num}: Invalid volume '{vol_raw}' for solution {label}.")
                    continue

                solution = {
                    "label": label,
                    "name": name,
                    "volume_mL": volume_mL
                }

                well["solutions"].append(solution)

            wells.append(well)

    return wells, errors 

In [4]:
# SETUP LOGGING------------------------------------------------------------------------------------

def record_event(strMetadataPath: str, name: str, temp: float | None = None):
    if os.path.exists(strMetadataPath):
        with open(strMetadataPath, 'r') as f:
            meta = json.load(f)

    else: 
        meta = {}
    
    events = meta.get("events", {})

    entry = {"time": datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
    if temp is not None:
        entry["temp_C"] = temp
    events[name] = entry

    meta["events"] = events
    with open(strMetadataPath, 'w') as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

    logging.info(f"Recorded event '{name}' in metadata.")

def record_ph_series(strMetadataPath: str, series):
    """
    Store the pH time series into metadata as a list of 
    {"time": timestamp, "pH": value}
    """ 
    if os.path.exists(strMetadataPath):
        with open(strMetadataPath, 'r') as f:
            meta = json.load(f)
    else: 
        meta = {}

    ph_list = []
    for ts, val in series:
        ph_list.append({"timestamp": ts, "pH": val})

    meta["pH_series"] = ph_list
    with open(strMetadataPath, 'w') as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

    logging.info(f"Recorded %d pH points in metadata.", len(series))

def record_experiment_data(strMetadataPath: str,
                           section: str,
                           key: str,
                           value):
    """
    Record a key-value pair into experimentData[section][key].
    Automatically converts tuples like ('A1', 10) into dicts.
    Handles lists from allocate_from_sources.
    """

    # Load metadata or create new one
    if os.path.exists(strMetadataPath):
        with open(strMetadataPath, 'r') as f:
            meta = json.load(f)
    else:
        meta = {}

    # Ensure base structure
    experimentData = meta.get("experimentData", {})
    experimentData.setdefault("deposition", {})
    experimentData.setdefault("characterization", {})

    # Validate section
    if section not in experimentData:
        raise ValueError(f"Unknown section '{section}'. Must be 'deposition' or 'characterization'.")

    # Convert tuples to dicts (for readability)
    def convert(item):
        if isinstance(item, tuple) and len(item) == 2:
            well, amount = item
            return {"well": well, "uL": amount}
        return item

    # If list of tuples → convert each
    if isinstance(value, list):
        value = [convert(v) for v in value]
    else:
        value = convert(value)

    # Store it
    experimentData[section][key] = value
    meta["experimentData"] = experimentData

    # Write metadata back
    with open(strMetadataPath, 'w') as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

    logging.info(f"Recorded {section}.{key} in metadata.")


# Create a log file with the same name as the script but with the extension '.log'
# get the path to the current directory
strWD = os.getcwd()
# get the name of this file
strLogFileName = "alan_workflow.log"
# split the file name and the extension
strLogFileName = os.path.splitext(strLogFileName)[0]
# add .log to the file name
strLogFileName = strLogFileName + ".log"
# join the log file name to the current directory
strLogFilePath = os.path.join(strWD, strLogFileName)

# Initialize logging
logging.basicConfig(
    level = logging.DEBUG,                                                      # Can be changed to logging.INFO to see less
    format = "%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler(strLogFilePath, mode="a"),
        logging.StreamHandler(sys.stdout),
    ],
)

# get the current time
strTime_start = datetime.now().strftime("%H:%M:%S")

# get the current date
strDate = datetime.now().strftime("%Y%m%d")

# Automatically get next run number from data folders created
data_path = "data"
subfolders = [int(f.name.split('_')[1]) for f in os.scandir(data_path) if f.is_dir() and f.name.startswith(strDate)]
if not subfolders:
    subfolders = [000]
intSuggestedRunNumber = max(subfolders)

### pin: shouldn't this be intSuggestedRunNumber + 1?

# Get number input form user
intRunNumber = intSuggestedRunNumber + 1

# make a string with the experimentID
strExperimentID = f"{strDate}_{intRunNumber:03}"
strWD = os.getcwd()

# make a new directory in the data folder to store the results
strExperimentPath = os.path.join(strWD, 'data', strExperimentID)
os.makedirs(strExperimentPath, exist_ok=True)

In [5]:
# Load experiment parameters
strParamsPath = strExperimentPath + "/experiment_params.csv"

# initialize params file 
if not os.path.exists(strParamsPath):
    os.makedirs(os.path.dirname(strParamsPath), exist_ok=True)
    header = [
        "well ID", "wellName", "temperature_C", "depositionCurrent_mA", "depositionTime_s"
    ]
    for label in ['A', 'B', 'C', 'D']:
        header += [
            f"solution {label} name",
            f"solution {label} volume_mL",
        ]

    with open(strParamsPath, "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=header)
        writer.writeheader()

        # write a single example row as a template
        writer.writerow({
            "wellName": "A1",
            "well ID": 1,
            "temperature_C": 25.0,
            "depositionCurrent_mA": -8.0,
            "depositionTime_s": 180.0,
            "solution A name": "KOH",
            "solution A volume_mL": 5.0,
        })

    print(f"Initialized params file: {strParamsPath}")
else:
    print(f"Params file already exists: {strParamsPath}")

paramsReady = (UI_cli.getNumberInput("Press 1 when experiment parameters are ready", min=1, max=1) == 1)

if not paramsReady:
    print("Please prepare the experiment parameters file and run again.")
    sys.exit(0)

# load experiment parameters from csv
wells, errors = load_experiment_csv(strParamsPath)
if errors: 
    print(f"Errors in {strParamsPath}:")
    for error in errors:
        print(" -", error)

print(f"Loaded experiment parameters at {strParamsPath}:")
for well in wells:
    print(f"Well {well['well_name']} (ID: {well['well_id']}):")
    print(f"  Temperature: {well['temperature_C']} C")
    print(f"  Deposition Current: {well['depositionCurrent_mA']} mA")
    print(f"  Deposition Time: {well['depositionTime_s']} s")
    print("  Solutions:")
    for sol in well['solutions']:
        print(f"    - {sol['label']}: {sol['name']} ({sol['volume_mL']} mL)")

sources_by_plate = {
    "strID_vialRack_4": {
        "A1": {"solution": "FeCl3", "remaining_uL": 15000},
        "A2": {"solution": "FeCl3", "remaining_uL": 15000},
        "A3": {"solution": "CrCl3", "remaining_uL": 15000},
        "A4": {"solution": "ZnCl2", "remaining_uL": 15000},
        "B1": {"solution": "KOH", "remaining_uL": 15000},
        "B2": {"solution": "KOH", "remaining_uL": 15000},
        "B3": {"solution": "KOH", "remaining_uL": 15000},
        "B4": {"solution": "KOH", "remaining_uL": 15000},
    }, 
    "strID_vialRack_7": {
        # "A1": {"solution": "Solution I", "remaining_uL": 15000},
        # "A2": {"solution": "Solution J", "remaining_uL": 15000},
        # "A3": {"solution": "Solution K", "remaining_uL": 15000},
        # "A4": {"solution": "Solution L", "remaining_uL": 15000},
        # "B1": {"solution": "Solution M", "remaining_uL": 15000},
        # "B2": {"solution": "Solution N", "remaining_uL": 15000},
        # "B3": {"solution": "Solution O", "remaining_uL": 15000},
        # "B4": {"solution": "Solution P", "remaining_uL": 15000},
    }
}
# make a variable to store the well in the reactor to be used
# strWell2Test = UI_cli.getAddressInput("Test Well Address", numRows=3, numCols=5)

# make a variable to store the next pipette tip location
intPipetteTipLoc = UI_cli.getNumberInput("Pipette Tip ID", min=1, max=96)


Initialized params file: c:\Users\sdl1_\Downloads\AC_OT2_Workflow-main\AC_OT2_Workflow-main\data\20251128_002/experiment_params.csv
Select a Press 1 when experiment parameters are ready (1-1):
Press 1 when experiment parameters are ready selected: 1
Loaded experiment parameters at c:\Users\sdl1_\Downloads\AC_OT2_Workflow-main\AC_OT2_Workflow-main\data\20251128_002/experiment_params.csv:
Well A1 (ID: 1):
  Temperature: 25.0 C
  Deposition Current: -8.0 mA
  Deposition Time: 180.0 s
  Solutions:
    - A: KOH (5.0 mL)
Select a Pipette Tip ID (1-96):
Pipette Tip ID selected: 1


In [3]:
# SETUP OPENTRONS PLATFORM-------------------------------------------------------------------------

# robotIP = "100.67.86.197"
# robotIP = "100.66.20.191"
# robotIP = "100.67.89.154"
robotIP = "192.168.0.107"

# initialize an the opentrons client
oc = opentronsClient(
    strRobotIP = robotIP
)


# -----LOAD OPENTRONS STANDARD LABWARE-----

    # -----LOAD OPENTRONS TIP RACK-----
# load opentrons tip rack in slot 1
strID_pipetteTipRack = oc.loadLabware(
    intSlot = 1,
    strLabwareName = 'opentrons_96_tiprack_1000ul'
)

# -----LOAD CUSTOM LABWARE-----

# get path to current directory
strCustomLabwarePath = os.getcwd()
# join "labware" folder to current directory
strCustomLabwarePath = os.path.join(strCustomLabwarePath, 'labware')

    # -----LOAD sonicator bath RACK-----

strCustomLabwarePath_temp = os.path.join(strCustomLabwarePath, 'nis_2_sonicator_bath.json')
# read json file
with open(strCustomLabwarePath_temp) as f:
    dicCustomLabware_temp = json.load(f)
# load custom labware in slot 2
strID_bath_2 = oc.loadCustomLabware(
    dicLabware = dicCustomLabware_temp,
    intSlot = 3
)

    # -----LOAD 25ml VIAL RACK-----
# join "nis_8_reservoir_25000ul.json" to labware directory
strCustomLabwarePath_temp = os.path.join(strCustomLabwarePath, 'nis_8_reservoir_25000ul.json')
# read json file
with open(strCustomLabwarePath_temp) as f:
    dicCustomLabware_temp = json.load(f)
# load custom labware in slot 2
strID_vialRack_4 = oc.loadCustomLabware(
    dicLabware = dicCustomLabware_temp,
    intSlot = 4
)

# load custom labware in slot 7
strID_vialRack_7 = oc.loadCustomLabware(
    dicLabware = dicCustomLabware_temp,
    intSlot = 7
)

    # -----LOAD NIS'S REACTOR-----
# join "nis_15_wellplate_3895ul.json" to labware directory
strCustomLabwarePath_temp = os.path.join(strCustomLabwarePath, 'nis_15_wellplate_3895ul.json')

# read json file
with open(strCustomLabwarePath_temp) as f:
    dicCustomLabware_temp = json.load(f)

strID_NISreactor = oc.loadCustomLabware(
    dicLabware = dicCustomLabware_temp,
    intSlot = 9
)

    # -----LOAD ELECTRODE TIP RACK-----
# join "nis_4_tiprack_1ul.json" to labware directory
strCustomLabwarePath_temp = os.path.join(strCustomLabwarePath, 'nistall_4_tiprack_1ul.json')

# read json file
with open(strCustomLabwarePath_temp) as f:
    dicCustomLabware_temp = json.load(f)

# load custom labware in slot 10
strID_electrodeTipRack = oc.loadCustomLabware(
    dicLabware = dicCustomLabware_temp,
    intSlot = 10
)


strCustomLabwarePath_temp = os.path.join(strCustomLabwarePath, 'nis_1_ph_probe_rack.json')

# read json file
with open(strCustomLabwarePath_temp) as f:
    dicCustomLabware_temp = json.load(f)

# load custom labware in slot 10
strID_phTipRack = oc.loadCustomLabware(
    dicLabware = dicCustomLabware_temp,
    intSlot = 11
)

# LOAD OPENTRONS STANDARD INSTRUMENTS--------------------------------------------------------------
# add pipette
oc.loadPipette(
    strPipetteName = 'p1000_single_gen2',
    strMount = 'right'

)

2025-11-28 11:15:38,720 [DEBUG] Starting new HTTP connection (1): 192.168.0.107:31950
2025-11-28 11:15:44,827 [DEBUG] http://192.168.0.107:31950 "POST /runs HTTP/1.1" 201 252
2025-11-28 11:15:44,828 [INFO] New run created with ID: 5a7b0100-557e-43e1-8fea-f4740c100942
2025-11-28 11:15:44,828 [INFO] Command URL: http://192.168.0.107:31950/runs/5a7b0100-557e-43e1-8fea-f4740c100942/commands
2025-11-28 11:15:44,829 [INFO] Loading labware: opentrons_96_tiprack_1000ul in slot: 1
2025-11-28 11:15:44,829 [DEBUG] Command: {"data": {"commandType": "loadLabware", "params": {"location": {"slotName": "1"}, "loadName": "opentrons_96_tiprack_1000ul", "namespace": "opentrons", "version": "1"}, "intent": "setup"}}
2025-11-28 11:15:44,830 [DEBUG] Starting new HTTP connection (1): 192.168.0.107:31950
2025-11-28 11:15:45,144 [DEBUG] http://192.168.0.107:31950 "POST /runs/5a7b0100-557e-43e1-8fea-f4740c100942/commands?waitUntilComplete=True HTTP/1.1" 201 14340
2025-11-28 11:15:45,145 [DEBUG] Response: {"data

In [5]:
# SETUP MQTT BROKER AND DEVICE CLIENTS------------------------------------------------------------
import time
from iot_mqtt import (
    PumpMQTT, UltraMQTT, HeatMQTT, PhMQTT, BioMQTT,
    start_broker_if_needed, stop_broker,
    ControllerBeacon, _best_effort_all_off
)

# ─────────────────────────────────────────────
# 1) Start broker if needed
# ─────────────────────────────────────────────
proc = start_broker_if_needed()   # No-op if already running

broker = "192.168.0.100"

# ─────────────────────────────────────────────
# 2) Controller beacon (heartbeat + ONLINE/OFFLINE)
# ─────────────────────────────────────────────
beacon = ControllerBeacon(
    broker=broker, port=1883,
    username="pyctl-controller", password="controller",
    client_id="pyctl-controller",
    status_topic="pyctl/status",
    heartbeat_topic="pyctl/heartbeat",
    heartbeat_interval=5.0,
    keepalive=30,
)
beacon.start()

# ─────────────────────────────────────────────
# 3) Device MQTT Clients
# ─────────────────────────────────────────────
pumps = PumpMQTT(
    broker=broker, username="pyctl-controller", password="controller",
    base_topic="pumps/01", client_id="pyctl-pumps"
)
ultra = UltraMQTT(
    broker=broker, username="pyctl-controller", password="controller",
    base_topic="ultra/01", client_id="pyctl-ultra"
)
heat = HeatMQTT(
    broker=broker, username="pyctl-controller", password="controller",
    base_topic="heat/01", client_id="pyctl-heat"
)
ph = PhMQTT(
    broker=broker, username="pyctl-controller", password="controller",
    base_topic="ph/01", client_id="pyctl-ph"
)
bio = BioMQTT(
    broker=broker, username="pyctl-controller", password="controller",
    base_topic="bio/01", client_id="pyctl-bio"
)

# ─────────────────────────────────────────────
# 4) Connect all devices
# ─────────────────────────────────────────────
pumps.start()
ultra.start()
heat.start()
ph.start()
bio.start()

time.sleep(1)  # allow MQTT connections to stabilize
logging.info("Connected to broker + all device clients.")    

2025-11-25 16:18:30,180 [INFO] [broker] Already listening on port 1883
2025-11-25 16:18:30,183 [INFO] [ctl] Connected -> 192.168.0.100:1883 (client_id=pyctl-controller-1125_1618-b3dd)
2025-11-25 16:18:30,183 [INFO] [pyctl-pumps-1125_1618-d820] Connecting to 192.168.0.100:1883 (attempt 1)...
2025-11-25 16:18:30,187 [INFO] [pyctl-ultra-1125_1618-343a] Connecting to 192.168.0.100:1883 (attempt 1)...
2025-11-25 16:18:30,187 [INFO] [pyctl-pumps-1125_1618-d820] Connected to 192.168.0.100:1883
2025-11-25 16:18:30,191 [INFO] [pyctl-heat-1125_1618-cf20] Connecting to 192.168.0.100:1883 (attempt 1)...
2025-11-25 16:18:30,191 [INFO] [pyctl-ultra-1125_1618-343a] Connected to 192.168.0.100:1883
2025-11-25 16:18:30,212 [INFO] [pyctl-ph-1125_1618-2d1e] Connecting to 192.168.0.100:1883 (attempt 1)...
2025-11-25 16:18:30,212 [INFO] [pyctl-heat-1125_1618-cf20] Connected to 192.168.0.100:1883
2025-11-25 16:18:30,216 [INFO] [pyctl-bio-1125_1618-fd96] Connecting to 192.168.0.100:1883 (attempt 1)...
2025-11

In [6]:
# Show live status from each node for a couple seconds
# pumps.status(seconds=2.0)
# time.sleep(1)
# ultra.status(seconds=2.0)
# time.sleep(1)
# heat.status(seconds=2.0)
# time.sleep(1)
# ph.status(seconds=2.0)
# time.sleep(1)
# bio.status(seconds=2.0)

In [7]:


# # START EXPERIMENT---------------------------------------------------------------------------------
oc.lights(True)

oc.homeRobot()

2025-11-25 16:18:31,266 [INFO] Lights On: true
2025-11-25 16:18:31,267 [DEBUG] Command: {"on": "true"}
2025-11-25 16:18:31,269 [DEBUG] Starting new HTTP connection (1): 192.168.0.107:31950
2025-11-25 16:18:31,289 [DEBUG] http://192.168.0.107:31950 "POST /robot/lights HTTP/1.1" 200 11
2025-11-25 16:18:31,290 [DEBUG] Response: {"on":true}
2025-11-25 16:18:31,291 [INFO] Light change successful.
2025-11-25 16:18:31,291 [INFO] Homing the robot
2025-11-25 16:18:31,292 [DEBUG] Command: {"target": "robot"}
2025-11-25 16:18:31,293 [DEBUG] Starting new HTTP connection (1): 192.168.0.107:31950
2025-11-25 16:18:54,694 [DEBUG] http://192.168.0.107:31950 "POST /robot/home HTTP/1.1" 200 27
2025-11-25 16:18:54,697 [DEBUG] Response: {"message":"Homing robot."}
2025-11-25 16:18:54,698 [INFO] Robot homed successfully.


In [ ]:
# Full experiment run setup--------------------------------------------------------------------------
for well in wells:
    i = wellNameToIndex(well["well_name"])
    # START EXPERIMENT---------------------------------------------------------------------------------
    strWell2Test = well["well_name"]
    logging.info(f"Starting experiment in well {strWell2Test} (index {i})")

    bioChannel = i

    totalVolume_uL = sum(int(sol["volume_mL"] * 1000) for sol in well["solutions"])
    depositionCurrent = well["depositionCurrent_mA"] / 1000.0  # convert to A
    depositionDuration = int(well["depositionTime_s"])

    well_path = os.path.join(strExperimentPath, strWell2Test)
    os.makedirs(well_path, exist_ok=True)

    deposition_path = os.path.join(well_path, 'deposition')
    characterization_path = os.path.join(well_path, 'characterization')

    os.makedirs(deposition_path, exist_ok=True)
    os.makedirs(characterization_path, exist_ok=True)

    # make a metadata file in the new directory
    strMetadataPath = os.path.join(well_path, f"metadata.json")

    # customize the note
    defaultNote = "General electrodeposition test"
    # userNote = input(f"Enter notes for this experiment (or press Enter for default: '{defaultNote}'): ").strip()
    # strNote = userNote if userNote else defaultNote
    strNote = defaultNote

    dicMetadata = {"date": strDate,
                "time": strTime_start,
                "runNumber": f"{intRunNumber:03}",
                "experimentID": strExperimentID,
                "wellName": strWell2Test,
                "status": "running",
                "notes": strNote}

    with open(strMetadataPath, 'w') as f:
        json.dump(dicMetadata, f)

    # start video recording
    rec = VideoRecorder(camera_index=2, out_path=os.path.join(well_path, "experiment.mp4"))
    rec.start()
    time.sleep(2)  # wait for 2 seconds to stabilize

    # log the start of the experiment
    logging.info(f"Experiment {strExperimentID} started in well {strWell2Test}!")

    # take picture at very start
    take_picture(oc, strID_NISreactor, strWell2Test, '1_start.jpg', well_path)
    logging.info("Taken start picture.")

    time.sleep(0.01)

    record_experiment_data(strMetadataPath, "deposition", "current", depositionCurrent)
    record_experiment_data(strMetadataPath, "deposition", "duration", depositionDuration)
    logging.info(f"Deposition parameters: current={depositionCurrent} mA, duration={depositionDuration} s")

    # fill the reactor well with solutions
    for i, sol in enumerate(well["solutions"]):
        solutionName = sol["name"]
        volume_mL = sol["volume_mL"]
        volume_uL = int(volume_mL * 1000)

        logging.info(f"Adding solution '{solutionName}' ({volume_mL} mL) to well {strWell2Test}.")

        needMixing = False
        if (len(well["solutions"]) > 1) and (i == len(well["solutions"]) - 1):
            needMixing = True
            
        # pick up pipette tip
        oc.moveToWell(
            strLabwareName = strID_pipetteTipRack,
            strWellName = getPipetteTipLocById(intPipetteTipLoc),
            strPipetteName = 'p1000_single_gen2',
            strOffsetStart = 'top',
            fltOffsetY = 1,
            intSpeed = 100
        )

        time.sleep(0.01)

        oc.pickUpTip(
            strLabwareName = strID_pipetteTipRack,
            strPipetteName = 'p1000_single_gen2',
            strWellName = getPipetteTipLocById(intPipetteTipLoc),
            fltOffsetY = 1
        )

        time.sleep(0.01)

        # fill the reactor well from vial rack
        fillWell_autoSource(
            opentronsClient = oc,
            sources_by_plate = sources_by_plate,
            solution_name= solutionName,
            strOffsetStart_from = 'bottom',
            strPipetteName = 'p1000_single_gen2',
            strLabwareName_to = strID_NISreactor,
            strWellName_to = strWell2Test,
            strOffsetStart_to = 'top',
            totalVolume_uL = volume_uL,
            fltOffsetX_from = 0,
            fltOffsetY_from = 0,
            fltOffsetZ_from = 8,
            fltOffsetX_to = -1,
            fltOffsetY_to = 0.5,
            fltOffsetZ_to = 0,
            intMoveSpeed = 100,
            needMixing = needMixing,
            experimentName = "deposition"
        )

        time.sleep(0.01)

        # drop pipette tip
        oc.dropTip(
            strLabwareName = strID_pipetteTipRack,
            strPipetteName = 'p1000_single_gen2',
            strWellName = getPipetteTipLocById(intPipetteTipLoc),
            strOffsetStart = 'bottom',
            fltOffsetY = 1,
            fltOffsetZ = 7
        )

        intPipetteTipLoc += 1


    time.sleep(0.01)

    # pick up ph probe
    oc.moveToWell(
        strLabwareName = strID_phTipRack,
        strWellName = 'A1',
        strPipetteName = 'p1000_single_gen2',
        strOffsetStart = 'top',
        fltOffsetZ = 0,
        intSpeed = 100
    )

    time.sleep(0.01)

    oc.pickUpTip(
        strLabwareName = strID_phTipRack,
        strPipetteName = 'p1000_single_gen2',
        strWellName = 'A1',
        fltOffsetZ = -10
    )
    
    time.sleep(0.01)

    # move to sonicator bath
    oc.moveToWell(
        strLabwareName = strID_bath_2,
        strWellName = 'A1',
        strPipetteName = 'p1000_single_gen2',
        strOffsetStart = 'top',
        fltOffsetX = 15,
        fltOffsetY = 0,
        fltOffsetZ = 0,
        intSpeed = 100
    )

    time.sleep(0.01)

    oc.moveToWell(
        strLabwareName = strID_bath_2,
        strWellName = 'A1',
        strPipetteName = 'p1000_single_gen2',
        strOffsetStart = 'top',
        fltOffsetX = 15,
        fltOffsetY = 0,
        fltOffsetZ = -30,
        intSpeed = 100
    )

    time.sleep(0.01)

    # turn on sonicator for 15 seconds
    ultra.on(2, 15000)  # ultrasonic ch1 ON for 15 s (auto-off)

    time.sleep(16)

    oc.moveToWell(
        strLabwareName = strID_bath_2,
        strWellName = 'A1',
        strPipetteName = 'p1000_single_gen2',
        strOffsetStart = 'top',
        fltOffsetX = 15,
        fltOffsetY = 0,
        fltOffsetZ = 0,
        intSpeed = 100
    )

    time.sleep(0.01)

    # move to reactor well 
    oc.moveToWell(strLabwareName = strID_NISreactor,
                strWellName = strWell2Test,
                strPipetteName = 'p1000_single_gen2',
                strOffsetStart = 'top',
                fltOffsetX = 15, 
                fltOffsetY = 0,
                fltOffsetZ = -20, # change this if ph head can't be immersed 
                intSpeed = 50)

    time.sleep(40)

    # Single read 
    # ph.oneshot(seconds=3.0) 

    # 20 seconds read, record per 2 seconds
    series = ph.watch_poll(interval_ms=2000, seconds=40.0, collect=True)
    print("Collected points:", len(series))
    for ts, val in series:
        print(ts, val)

    logging.info(f"pH measurement series: {series}")
    record_ph_series(strMetadataPath, series)

    time.sleep(0.01)

    # move to sonicator bath
    oc.moveToWell(
        strLabwareName = strID_bath_2,
        strWellName = 'A1',
        strPipetteName = 'p1000_single_gen2',
        strOffsetStart = 'top',
        fltOffsetX = 15,
        fltOffsetY = 0,
        fltOffsetZ = 0,
        intSpeed = 100
    )

    time.sleep(0.01)

    oc.moveToWell(
        strLabwareName = strID_bath_2,
        strWellName = 'A1',
        strPipetteName = 'p1000_single_gen2',
        strOffsetStart = 'top',
        fltOffsetX = 15,
        fltOffsetY = 0,
        fltOffsetZ = -30,
        intSpeed = 100
    )

    time.sleep(1)

    # turn on sonicator for 15 seconds
    ultra.on(2, 15000)  # ultrasonic ch1 ON for 15 s (auto-off)

    time.sleep(16)

    oc.moveToWell(
        strLabwareName = strID_bath_2,
        strWellName = 'A1',
        strPipetteName = 'p1000_single_gen2',
        strOffsetStart = 'top',
        fltOffsetX = 15,
        fltOffsetY = 0,
        fltOffsetZ = 0,
        intSpeed = 100
    )

    time.sleep(0.01)

    # drop ph probe
    oc.moveToWell(
        strLabwareName = strID_phTipRack,
        strWellName = 'A1',
        strPipetteName = 'p1000_single_gen2',
        strOffsetStart = 'top',
        fltOffsetZ = 0,
        intSpeed = 100
    )

    time.sleep(0.01)

    oc.dropTip(strLabwareName = strID_phTipRack,
                strPipetteName = 'p1000_single_gen2',
                boolDropInDisposal = False,
                strWellName = 'A1',
                fltOffsetZ = 6,
                strOffsetStart = "bottom")

    time.sleep(0.01)

    oc.moveToWell(
        strLabwareName = strID_phTipRack,
        strWellName = 'A1',
        strPipetteName = 'p1000_single_gen2',
        strOffsetStart = 'top',
        fltOffsetZ = 50,
        intSpeed = 100
    )

    # pick up electrode tip
    oc.moveToWell(
        strLabwareName = strID_electrodeTipRack,
        strWellName = 'A2',
        strPipetteName = 'p1000_single_gen2',
        strOffsetStart = 'top',
        fltOffsetX = 0.6,
        fltOffsetY = 0.5,
        fltOffsetZ = 3,
        intSpeed = 100
    )

    time.sleep(0.01)

    oc.pickUpTip(
        strLabwareName = strID_electrodeTipRack,
        strPipetteName = 'p1000_single_gen2',
        strWellName = 'A2',
        fltOffsetX = 0.6,
        fltOffsetY = 0.5
    )

    time.sleep(0.01)

    # move to sonicator bath
    oc.moveToWell(
        strLabwareName = strID_bath_2,
        strWellName = 'A1',
        strPipetteName = 'p1000_single_gen2',
        strOffsetStart = 'bottom',
        fltOffsetZ = 50,
        intSpeed = 100
    )

    time.sleep(1)

    # turn on sonicator for 15 seconds
    ultra.on(2, 15000)  # ultrasonic ch1 ON for 15 s (auto-off)

    time.sleep(16)

    # move to reactor well 
    oc.moveToWell(strLabwareName = strID_NISreactor,
                strWellName = strWell2Test,
                strPipetteName = 'p1000_single_gen2',
                strOffsetStart = 'top',
                fltOffsetX = 0.5,
                fltOffsetY = 0.5,
                fltOffsetZ = 5,
                intSpeed = 50)

    time.sleep(0.01)

    oc.moveToWell(strLabwareName = strID_NISreactor,
                strWellName = strWell2Test,
                strPipetteName = 'p1000_single_gen2',
                strOffsetStart = 'top',
                fltOffsetX = 0.5,
                fltOffsetY = 0.5,
                fltOffsetZ = -25,
                intSpeed = 50)

    time.sleep(0.01)
    
    # RUN ELECTRODEPOSITION EXPERIMENT---------------------------------------------
    # -----CP-----
    # make deposition step
    cpStep_deposition = CPStep(
        current=depositionCurrent,
        duration=depositionDuration,
        vs_initial=False
    )

    cpParams_deposition = CPParams(
        record_every_dT=0.1,
        record_every_dE=0.05,
        n_cycles=0,
        steps=[cpStep_deposition],
        I_range=I_RANGE.I_RANGE_10mA
    )

    cpTech_deposition = CPTechnique(cpParams_deposition)

    # -----OCV-----
    # create OCV parameters
    ocvParams_1mins = OCVParams(
        rest_time_T = 30,
        record_every_dT = 0.5,
        record_every_dE = 10,
        E_range = E_RANGE.E_RANGE_2_5V,
        bandwidth = BANDWIDTH.BW_5,
        )

    # create OCV technique
    ocvTech_1mins = OCVTechnique(ocvParams_1mins)

    boolTryToConnect = True
    intAttempts_temp = 0
    intMaxAttempts = 5


    # -----LSV-----

    # create LP steps for testing OER performance
    Ei_lsv = LPStep(
        voltage_scan=0.5,
        scan_rate=0.010,
        vs_initial_scan=False
    )
    El_lsv = LPStep(
        voltage_scan=-5.0,
        scan_rate=0.010,
        vs_initial_scan=False
    )
    # create LP parameters for LPR with OCP
    lpParams_lsv_wOCP = LPParams(
        record_every_dEr = 0.01,
        rest_time_T = 5,
        record_every_dTr = 0.5,
        Ei = Ei_lsv,
        El = El_lsv,
        record_every_dE = 0.001,
        average_over_dE = True,
        begin_measuring_I = 0.5,
        end_measuring_I = 1,
        # I_range = I_RANGE.I_RANGE_100mA,
        # E_range = E_RANGE.E_RANGE_2_5V
        I_range = I_RANGE.I_RANGE_100mA,
        E_range = E_RANGE.E_RANGE_10V
    )

    # create a technique for LPR with OCP
    lpTech_lsv_wOCP = LPTechnique(lpParams_lsv_wOCP)

    # initialize an empty dataframe to store the results
    dfData = pd.DataFrame()
    # initialize a counter to keep track of the technique index
    intID_tech = 0
    # initialize a counter to keep track of the number to add to technique index 
    # (to account for multiple processes)
    intID_tech_add = 0
    # initialize a string to keep track of the current technique
    strCurrentTechnique = ''

    boolNewTechnique = False
    boolAdd1ToTechIndex = False
    boolFirstTechnique = True

    fltTime_prev = 0
    fltTime_curr = 0

    logging.info("Starting electrodeposition experiment!")

    t = heat.get_base_temp(1, timeout_s=5.0)
    record_event(strMetadataPath, "before_deposition", temp=t)

    logging.info("Temperature before electrodeposition experiment: %s", t)

    bio.on(bioChannel)

    logging.info("Biologic powered on.")

    while boolTryToConnect and intAttempts_temp < intMaxAttempts:
        logging.info(f"Attempting to connect to the Biologic: {intAttempts_temp+1} / {intMaxAttempts}")

        try:
            # run all techniques
            with connect('USB0') as bl:
                channel = bl.get_channel(1)

                # run all techniques
                runner = channel.run_techniques([
                    ocvTech_1mins,
                    # lpTech_lsv_wOCP,
                    cpTech_deposition,
                    ocvTech_1mins
                ])
        
                for data_temp in runner:

                    if boolFirstTechnique:
                        # down select the string to only that which is inside the single quotes 
                        strCurrentTechnique = str(type(data_temp.data))
                        # down select the string to only that which is inside the single quotes
                        strCurrentTechnique = strCurrentTechnique.split("'")[1]
                        # down select the string to only that which is after the second to last period
                        strCurrentTechnique = strCurrentTechnique.split(".")[-2]
                        boolFirstTechnique = False

                    # check if the technique index is not the same as the previous technique index
                    if data_temp.tech_index != intID_tech:
                        boolNewTechnique = True
                    
                    if 'process_index' in data_temp.data.to_json(): 
                        dfData_temp = pd.DataFrame(data_temp.data.process_data.to_json(), index=[0])
                    else:
                        dfData_temp = pd.DataFrame(data_temp.data.to_json(), index=[0])
                        # if the time is available in the data
                        if 'time' in data_temp.data.to_json():
                            fltTime_prev = fltTime_curr
                            fltTime_curr = float(data_temp.data.to_json()['time'])
                        # if the previous time is greater than the current time but the technique id is the sam , then a new technique is being run
                        if (fltTime_prev-2 > fltTime_curr) and (data_temp.tech_index == intID_tech):
                            boolAdd1ToTechIndex = True
                            boolNewTechnique = True

                    if boolNewTechnique:
                        # the data coming in is from a new technique - save the previous data to a csv
                        dfData.to_csv(os.path.join(well_path,'deposition', f'{strExperimentID}_{intID_tech+intID_tech_add}_{strCurrentTechnique}.csv'))
                        # reinitialize the dataframe
                        dfData = pd.DataFrame()
                        # reset the boolean
                        boolNewTechnique = False

                        if boolAdd1ToTechIndex:
                            intID_tech_add += 1
                            boolAdd1ToTechIndex = False

                        # set the technique index to the current technique index
                        intID_tech = data_temp.tech_index
                        # update the current technique
                        strCurrentTechnique = str(type(data_temp.data))
                        # down select the string to only that which is inside the single quotes
                        strCurrentTechnique = strCurrentTechnique.split("'")[1]
                        # down select the string to only that which is after the second to last period
                        strCurrentTechnique = strCurrentTechnique.split(".")[-2] 

                    # log the data
                    logging.info(data_temp)
                    # add the data to the dataframe
                    dfData = pd.concat([dfData, dfData_temp], ignore_index=True)
                else:
                    time.sleep(1)

                # break the loop - successful connection
                boolTryToConnect = False
                # save the final data to a csv
                dfData.to_csv(os.path.join(well_path, 'deposition', f'{strExperimentID}_{intID_tech+intID_tech_add}_{strCurrentTechnique}.csv'))

        except Exception as e:
            logging.error(f"Failed to connect to the Biologic: {e}")
            logging.info(f"Attempting again in 30 seconds")
            time.sleep(30)
            intAttempts_temp += 1

    t = heat.get_base_temp(1, timeout_s=5.0)
    record_event(strMetadataPath, "after_deposition", temp=t)
    logging.info("Temperature after electrodeposition experiment: %s", t)

    # log the end of the experiment
    logging.info("End of electrodeposition experiment!")

    with open(strMetadataPath, 'r') as f:
        existing_data = json.load(f)

    # Update or add fields
    existing_data['status'] = "completed"
    existing_data['time'] = datetime.now().strftime("%H:%M:%S")

    # Write back to file
    with open(strMetadataPath, 'w') as f:
        json.dump(existing_data, f, indent=4)

    bio.off(bioChannel)

    # experiment completed

    # move to sonicator bath
    oc.moveToWell(
        strLabwareName = strID_bath_2,
        strWellName = 'A1',
        strPipetteName = 'p1000_single_gen2',
        strOffsetStart = 'top',
        fltOffsetZ = -40,
        intSpeed = 100
    )

    time.sleep(1)

    ultra.on(2, 15000)  # ultrasonic ch1 ON for 15 s (auto-off)
    time.sleep(16)

    # drop electrode tip
    oc.moveToWell(
        strLabwareName = strID_electrodeTipRack,
        strWellName = 'A2',
        strPipetteName = 'p1000_single_gen2',
        strOffsetStart = 'top',
        fltOffsetX = 0.6,
        fltOffsetY = 0.5,
        fltOffsetZ = 3,
        intSpeed = 100
    )

    time.sleep(0.01)

    oc.dropTip(strLabwareName = strID_electrodeTipRack,
                strPipetteName = 'p1000_single_gen2',
                boolDropInDisposal = False,
                strWellName = 'A2',
                fltOffsetX = 0.6,
                fltOffsetY = 0.5,
                fltOffsetZ = 6,
                strOffsetStart = "bottom")

    time.sleep(0.01)
    
    oc.moveToWell(
        strLabwareName=strID_electrodeTipRack,
        strWellName='A2',
        strPipetteName="p1000_single_gen2",
        strOffsetStart='top',
        fltOffsetX=0.5,
        fltOffsetY=0.5,
        fltOffsetZ=50,
        intSpeed=50
    )

    time.sleep(0.01)

    washReactor(oc,
                strID_NISreactor,
                strWell2Test,
                '2_pre_deposition_wash.jpg',
                '3_post_deposition_wash.jpg',
                pumps)
        
    time.sleep(0.01)

    # pick up new pipette tip
    oc.moveToWell(
        strLabwareName = strID_pipetteTipRack,
        strWellName = getPipetteTipLocById(intPipetteTipLoc),
        strPipetteName = 'p1000_single_gen2',
        strOffsetStart = 'top',
        fltOffsetY = 1,
        intSpeed = 100
    )

    time.sleep(0.01)

    oc.pickUpTip(
        strLabwareName = strID_pipetteTipRack,
        strPipetteName = 'p1000_single_gen2',
        strWellName = getPipetteTipLocById(intPipetteTipLoc),
        fltOffsetY = 1
    )

    time.sleep(0.01)

    # fill the reactor well from vial rack
    fillWell_autoSource(
        opentronsClient = oc,
        sources_by_plate = sources_by_plate,
        solution_name= "KOH",
        strOffsetStart_from = 'bottom',
        strPipetteName = 'p1000_single_gen2',
        strLabwareName_to = strID_NISreactor,
        strWellName_to = strWell2Test,
        strOffsetStart_to = 'top',
        totalVolume_uL = 5000,
        fltOffsetX_from = 0,
        fltOffsetY_from = 0,
        fltOffsetZ_from = 8,
        fltOffsetX_to = -1,
        fltOffsetY_to = 0.5,
        fltOffsetZ_to = 0,
        intMoveSpeed = 100,
        needMixing = False, 
        experimentName = "characterization"
    )

    time.sleep(0.01)

    # drop pipette tip
    oc.dropTip(
        strLabwareName = strID_pipetteTipRack,
        strPipetteName = 'p1000_single_gen2',
        strWellName = getPipetteTipLocById(intPipetteTipLoc),
        strOffsetStart = 'bottom',
        fltOffsetY = 1,
        fltOffsetZ = 7
    )

    intPipetteTipLoc += 1

    time.sleep(0.01)

    time.sleep(0.01)

    # pick up electrode tip
    oc.moveToWell(
        strLabwareName = strID_electrodeTipRack,
        strWellName = 'A2',
        strPipetteName = 'p1000_single_gen2',
        strOffsetStart = 'top',
        fltOffsetX = 0.6,
        fltOffsetY = 0.5,
        fltOffsetZ = 3,
        intSpeed = 100
    )

    time.sleep(0.01)

    oc.pickUpTip(
        strLabwareName = strID_electrodeTipRack,
        strPipetteName = 'p1000_single_gen2',
        strWellName = 'A2',
        fltOffsetX = 0.6,
        fltOffsetY = 0.5
    )

    time.sleep(0.01)

    # move to sonicator bath
    oc.moveToWell(
        strLabwareName = strID_bath_2,
        strWellName = 'A1',
        strPipetteName = 'p1000_single_gen2',
        strOffsetStart = 'top',
        fltOffsetZ = -40,
        intSpeed = 100
    )

    # turn on sonicator for 15 seconds
    ultra.on(2, 15000)  # ultrasonic ch1 ON for 15 s (auto-off)
    time.sleep(16)

    # move to reactor well 
    oc.moveToWell(strLabwareName = strID_NISreactor,
                strWellName = strWell2Test,
                strPipetteName = 'p1000_single_gen2',
                strOffsetStart = 'top',
                fltOffsetX = 0.5,
                fltOffsetY = 0.5,
                fltOffsetZ = 5,
                intSpeed = 50)

    time.sleep(0.01)

    oc.moveToWell(strLabwareName = strID_NISreactor,
                strWellName = strWell2Test,
                strPipetteName = 'p1000_single_gen2',
                strOffsetStart = 'top',
                fltOffsetX = 0.5,
                fltOffsetY = 0.5,
                fltOffsetZ = -22,
                intSpeed = 50)
    
    time.sleep(0.01)

    # run characterization experiments
    # -----OCV-----
    # create OCV parameters
    ocvParams_10sec = OCVParams(
        rest_time_T = 10,
        record_every_dT = 0.5,
        record_every_dE = 10,
        E_range = E_RANGE.E_RANGE_10V,
        bandwidth = BANDWIDTH.BW_5,
        )

    # create OCV technique
    ocvTech_10sec = OCVTechnique(ocvParams_10sec)


    # -----PEIS----- under OER & above OER
    applied_ocv = 0.01
    # applied_ocv_OER = 0.7

    peisParams_noOER = PEISParams(
        vs_initial = False,
        initial_voltage_step = 0.0,
        duration_step = 60,
        record_every_dT = 0.5,
        record_every_dI = 0.01,
        final_frequency = 1,
        initial_frequency = 200000,
        sweep = SweepMode.Logarithmic,
        amplitude_voltage = applied_ocv,
        frequency_number = 60,
        average_n_times = 2,
        correction = False,
        wait_for_steady = 0.1,
        bandwidth = BANDWIDTH.BW_5,
        E_range = E_RANGE.E_RANGE_2_5V
        )

    peisParams_OER = PEISParams(
        vs_initial = False,
        initial_voltage_step = 0.7,
        duration_step = 60,
        record_every_dT = 0.5,
        record_every_dI = 0.01,
        final_frequency = 1,
        initial_frequency = 200000,
        sweep = SweepMode.Logarithmic,
        amplitude_voltage = applied_ocv,
        frequency_number = 60,
        average_n_times = 2,
        correction = False,
        wait_for_steady = 0.1,
        bandwidth = BANDWIDTH.BW_5,
        E_range = E_RANGE.E_RANGE_2_5V
        )

    # create PEIS technique
    peisTech_noOER = PEISTechnique(peisParams_noOER)
    peisTech_OER = PEISTechnique(peisParams_OER)

    # -----CV-----
    # create CV steps -- active materials
    Ei_active = CVStep(
        voltage = 1.25,
        scan_rate = 0.005,
        vs_initial = False
    )
    E1_active = CVStep(
        voltage = 1.3,
        scan_rate = 0.005,
        vs_initial = False
    )
    E2_active = CVStep(
        voltage = 1.25,
        scan_rate = 0.005,
        vs_initial = False
    )
    Ef_active = CVStep(
        voltage = 1.25,
        scan_rate = 0.005,
        vs_initial = False
    )

    cvParams_active = CVParams(
        record_every_dE = 0.01,
        average_over_dE = True,
        n_cycles = 60,
        begin_measuring_i = 0.5,
        end_measuring_i = 1,
        Ei = Ei_active,
        E1 = E1_active,
        E2 = E2_active,
        Ef = Ef_active,
        bandwidth = BANDWIDTH.BW_5,
        I_range = I_RANGE.I_RANGE_100mA
    )

    cvTech_active = CVTechnique(cvParams_active)


    # -----CV----- check the redox peaks 
    Ei_active = CVStep(
        voltage = -0.8,
        scan_rate = 0.025,
        vs_initial = False
    )
    E1_active = CVStep(
        voltage = 1.0,
        scan_rate = 0.025,
        vs_initial = False
    )
    E2_active = CVStep(
        voltage = -0.8,
        scan_rate = 0.025,
        vs_initial = False
    )
    Ef_active = CVStep(
        voltage = 1.0,
        scan_rate = 0.025,
        vs_initial = False
    )

    cvParams_redox = CVParams(
        record_every_dE = 0.01,
        average_over_dE = True,
        n_cycles = 1,
        begin_measuring_i = 0.5,
        end_measuring_i = 1,
        Ei = Ei_active,
        E1 = E1_active,
        E2 = E2_active,
        Ef = Ef_active,
        bandwidth = BANDWIDTH.BW_5,
        I_range = I_RANGE.I_RANGE_100mA
    )

    cvTech_redox = CVTechnique(cvParams_redox)

    # -----CVA----- - CV with different sweeping speed
    # create CV steps
    Ei_20 = CVStep(
        voltage = -0.05,
        scan_rate = 0.02,
        vs_initial = True # RUNZE CHANGES
    )
    E1_20 = CVStep(
        voltage = 0.05,
        scan_rate = 0.02,
        vs_initial = True
    )
    E2_20 = CVStep(
        voltage = -0.05,
        scan_rate = 0.02,
        vs_initial = True
    )
    Ef_20 = CVStep(
        voltage = -0.05,
        scan_rate = 0.02,
        vs_initial = True
    )

    cvParams_20 = CVParams(
        record_every_dE = 0.001,
        average_over_dE = True,
        n_cycles = 3,
        begin_measuring_i = 0.5,
        end_measuring_i = 1,
        Ei = Ei_20,
        E1 = E1_20,
        E2 = E2_20,
        Ef = Ef_20,
        bandwidth = BANDWIDTH.BW_5,
        I_range = I_RANGE.I_RANGE_10mA
    )

    cvTech_20 = CVTechnique(cvParams_20)

    # create CV steps
    Ei_40 = CVStep(
        voltage = 0.0,
        scan_rate = 0.04,
        vs_initial = True 
    )
    E1_40 = CVStep(
        voltage = 0.10,
        scan_rate = 0.04,
        vs_initial = True
    )
    E2_40 = CVStep(
        voltage = -0.00,
        scan_rate = 0.04,
        vs_initial = True
    )
    Ef_40 = CVStep(
        voltage = -0.00,
        scan_rate = 0.04,
        vs_initial = True
    )

    cvParams_40 = CVParams(
        record_every_dE = 0.001,
        average_over_dE = True,
        n_cycles = 3,
        begin_measuring_i = 0.5,
        end_measuring_i = 1,
        Ei = Ei_40,
        E1 = E1_40,
        E2 = E2_40,
        Ef = Ef_40,
        bandwidth = BANDWIDTH.BW_5,
        I_range = I_RANGE.I_RANGE_10mA
    )

    cvTech_40 = CVTechnique(cvParams_40)

    # create CV steps
    Ei_60 = CVStep(
        voltage = -0.00,
        scan_rate = 0.06,
        vs_initial = True 
    )
    E1_60 = CVStep(
        voltage = 0.10,
        scan_rate = 0.06,
        vs_initial = True
    )
    E2_60 = CVStep(
        voltage = -0.00,
        scan_rate = 0.06,
        vs_initial = True
    )
    Ef_60 = CVStep(
        voltage = -0.00,
        scan_rate = 0.06,
        vs_initial = True
    )

    cvParams_60 = CVParams(
        record_every_dE = 0.001,
        average_over_dE = True,
        n_cycles = 3,
        begin_measuring_i = 0.5,
        end_measuring_i = 1,
        Ei = Ei_60,
        E1 = E1_60,
        E2 = E2_60,
        Ef = Ef_60,
        bandwidth = BANDWIDTH.BW_5,
        I_range = I_RANGE.I_RANGE_10mA
    )

    cvTech_60 = CVTechnique(cvParams_60)

    # create CV steps
    Ei_80 = CVStep(
        voltage = -0.00,
        scan_rate = 0.08,
        vs_initial = True 
    )
    E1_80 = CVStep(
        voltage = 0.10,
        scan_rate = 0.08,
        vs_initial = True
    )
    E2_80 = CVStep(
        voltage = -0.00,
        scan_rate = 0.08,
        vs_initial = True
    )
    Ef_80 = CVStep(
        voltage = -0.00,
        scan_rate = 0.08,
        vs_initial = True
    )

    cvParams_80 = CVParams(
        record_every_dE = 0.001,
        average_over_dE = True,
        n_cycles = 3,
        begin_measuring_i = 0.5,
        end_measuring_i = 1,
        Ei = Ei_80,
        E1 = E1_80,
        E2 = E2_80,
        Ef = Ef_80,
        bandwidth = BANDWIDTH.BW_5,
        I_range = I_RANGE.I_RANGE_10mA
    )

    cvTech_80 = CVTechnique(cvParams_80)

    # create CV steps
    Ei_100 = CVStep(
        voltage = -0.00,
        scan_rate = 0.1,
        vs_initial = True 
    )
    E1_100 = CVStep(
        voltage = 0.10,
        scan_rate = 0.1,
        vs_initial = True
    )
    E2_100 = CVStep(
        voltage = -0.00,
        scan_rate = 0.1,
        vs_initial = True
    )
    Ef_100 = CVStep(
        voltage = -0.00,
        scan_rate = 0.1,
        vs_initial = True
    )

    cvParams_100 = CVParams(
        record_every_dE = 0.001,
        average_over_dE = True,
        n_cycles = 3,
        begin_measuring_i = 0.5,
        end_measuring_i = 1,
        Ei = Ei_100,
        E1 = E1_100,
        E2 = E2_100,
        Ef = Ef_100,
        bandwidth = BANDWIDTH.BW_5,
        I_range = I_RANGE.I_RANGE_10mA
    )

    cvTech_100 = CVTechnique(cvParams_100)


    # ----- CV ----- stability test
    # create CV steps
    Ei_stability = CVStep(
        voltage = 0.6,
        scan_rate = 0.025,
        vs_initial = False 
    )
    E1_stability = CVStep(
        voltage = 1.00,
        scan_rate = 0.025,
        vs_initial = False
    )
    E2_stability = CVStep(
        voltage = 0.6,
        scan_rate = 0.025,
        vs_initial = False
    )
    Ef_stability = CVStep(
        voltage = 1.00,
        scan_rate = 0.025,
        vs_initial = False
    )

    cvParams_stability = CVParams(
        record_every_dE = 0.01,
        average_over_dE = True,
        # n_cycles = 50,
        n_cycles = 10,
        begin_measuring_i = 0.5,
        end_measuring_i = 1,
        Ei = Ei_stability,
        E1 = E1_stability,
        E2 = E2_stability,
        Ef = Ef_stability,
        bandwidth = BANDWIDTH.BW_5,
        I_range = I_RANGE.I_RANGE_10mA
    )

    cvTech_stability = CVTechnique(cvParams_stability)



    # -----LSV-----

    # create LP steps for testing OER performance
    Ei_lsv = LPStep(
        voltage_scan=-1.5,
        scan_rate=0.005,
        vs_initial_scan=False
    )
    El_lsv = LPStep(
        voltage_scan=1,
        scan_rate=0.005,
        vs_initial_scan=False
    )
    # create LP parameters for LPR with OCP
    lpParams_lsv_wOCP = LPParams(
        record_every_dEr = 0.01,
        rest_time_T = 5,
        record_every_dTr = 0.5,
        Ei = Ei_lsv,
        El = El_lsv,
        record_every_dE = 0.001,
        average_over_dE = True,
        begin_measuring_I = 0.5,
        end_measuring_I = 1,
        I_range = I_RANGE.I_RANGE_100mA,
        E_range = E_RANGE.E_RANGE_2_5V
    )

    # create a technique for LPR with OCP
    lpTech_lsv_wOCP = LPTechnique(lpParams_lsv_wOCP)



    # this is fucking stupid - fix it in the future -- not stupid, its ok!
    def run_and_save_characterization(
        strExperimentID,
        well_path,
        techniques_list,
        intGlobalTechID=0,  # for multiple batch
        intMaxAttempts=3,
        usb_port='USB0',
        channel_id=1
    ):
        """
        Run Biologic techniques in batches and save each technique's data as a separate CSV,
        with increasing global numbering across batches.
        """
        boolTryToConnect = True
        intAttempts_temp = 0

        boolFirstTechnique = True
        boolNewTechnique = False
        boolAdd1ToTechIndex = False

        intID_tech = None
        fltTime_prev = 0.0
        fltTime_curr = 0.0
        dfData = pd.DataFrame()
        strCurrentTechnique = ''

        while boolTryToConnect and intAttempts_temp < intMaxAttempts:
            logging.info(f"Attempting to connect to Biologic: {intAttempts_temp + 1} / {intMaxAttempts}")

            try:
                with connect(usb_port) as bl:
                    channel = bl.get_channel(channel_id)
                    runner = channel.run_techniques(techniques_list)

                    for data_temp in runner:
                        if boolFirstTechnique:
                            strCurrentTechnique = str(type(data_temp.data)).split("'")[1].split(".")[-2]
                            boolFirstTechnique = False

                        # detect whether a tech is new
                        if intID_tech is None or data_temp.tech_index != intID_tech:
                            boolNewTechnique = True

                        # load data
                        if 'process_index' in data_temp.data.to_json():
                            dfData_temp = pd.DataFrame(data_temp.data.process_data.to_json(), index=[0])
                        else:
                            dfData_temp = pd.DataFrame(data_temp.data.to_json(), index=[0])

                            if 'time' in data_temp.data.to_json():
                                fltTime_prev = fltTime_curr
                                fltTime_curr = float(data_temp.data.to_json()['time'])

                            if (fltTime_prev - 2 > fltTime_curr) and (data_temp.tech_index == intID_tech):
                                boolAdd1ToTechIndex = True
                                boolNewTechnique = True

                        if boolNewTechnique:
                            # save the last tech if current tech is not new
                            if not dfData.empty:
                                filename = f'{strExperimentID}_{intGlobalTechID}_{strCurrentTechnique}.csv'
                                filepath = os.path.join(well_path, 'characterization', filename)
                                dfData.to_csv(filepath, index=False)
                                logging.info(f"Saved: {filepath}")
                                intGlobalTechID += 1  

                            dfData = pd.DataFrame()
                            boolNewTechnique = False
                            boolAdd1ToTechIndex = False

                            intID_tech = data_temp.tech_index
                            strCurrentTechnique = str(type(data_temp.data)).split("'")[1].split(".")[-2]

                        logging.info(data_temp)
                        dfData = pd.concat([dfData, dfData_temp], ignore_index=True)

                    # save the last 
                    if not dfData.empty:
                        filename = f'{strExperimentID}_{intGlobalTechID}_{strCurrentTechnique}.csv'
                        filepath = os.path.join(well_path, 'characterization', filename)
                        dfData.to_csv(filepath, index=False)
                        logging.info(f"Saved final: {filepath}")
                        intGlobalTechID += 1

                    boolTryToConnect = False

            except Exception as e:
                logging.error(f"Failed to connect to the Biologic: {e}")
                logging.info("Retrying in 50 seconds...")
                time.sleep(50)
                intAttempts_temp += 1

        logging.info("Finished one batch of characterization.")
        return intGlobalTechID  # return id

    intGlobalTechID = 0

    t = heat.get_base_temp(1, timeout_s=5.0)
    record_event(strMetadataPath, "before_characterization", temp=t)
    logging.info("Temperature before characterization experiment: %s", t)

    bio.on(bioChannel)

    # First job request
    intGlobalTechID = run_and_save_characterization(
        strExperimentID, well_path,
        [ocvTech_10sec, cvTech_20, cvTech_40, cvTech_60,
        cvTech_80, cvTech_100, peisTech_noOER, peisTech_OER,
        cvTech_redox, ocvTech_10sec,lpTech_lsv_wOCP],
        intGlobalTechID
    )

    t = heat.get_base_temp(1, timeout_s=5.0)
    record_event(strMetadataPath, "after_characterization", temp=t)
    logging.info("Temperature after characterization experiment: %s", t)

    bio.off(bioChannel)
    # log the end of the experiment
    logging.info("End of characterization experiment!")

    # expriment completed

    time.sleep(0.01)

    # move to sonicator bath
    oc.moveToWell(
        strLabwareName = strID_bath_2,
        strWellName = 'A1',
        strPipetteName = 'p1000_single_gen2',
        strOffsetStart = 'top',
        fltOffsetZ = -40,
        intSpeed = 100
    )

    time.sleep(1)

    ultra.on(2, 15000)  # ultrasonic ch1 ON for 15 s (auto-off)

    time.sleep(16)

    # drop electrode tip
    oc.moveToWell(
        strLabwareName = strID_electrodeTipRack,
        strWellName = 'A2',
        strPipetteName = 'p1000_single_gen2',
        strOffsetStart = 'top',
        fltOffsetX = 0.6,
        fltOffsetY = 0.5,
        fltOffsetZ = 3,
        intSpeed = 100
    )

    time.sleep(0.01)

    oc.dropTip(strLabwareName = strID_electrodeTipRack,
                strPipetteName = 'p1000_single_gen2',
                boolDropInDisposal = False,
                strWellName = 'A2',
                fltOffsetX = 0.6,
                fltOffsetY = 0.5,
                fltOffsetZ = 6,
                strOffsetStart = "bottom")

    time.sleep(0.01)

    oc.moveToWell(
        strLabwareName=strID_electrodeTipRack,
        strWellName='A2',
        strPipetteName="p1000_single_gen2",
        strOffsetStart='top',
        fltOffsetX=0.5,
        fltOffsetY=0.5,
        fltOffsetZ=50,
        intSpeed=50
    )
    
    time.sleep(0.01)

    # wash reactor
    washReactor(oc,
                strID_NISreactor,
                strWell2Test,
                '4_pre_characterization_wash.jpg',
                '5_post_characterization_wash.jpg',
                pumps)

    time.sleep(0.01)
    
    oc.homeRobot()

    rec.stop()

    logging.info("Finished electrodeposition and characterization experiment for well %s.", strWell2Test)

2025-11-27 14:17:37,728 [INFO] Starting experiment in well A1 (index 1)


RuntimeError: Could not open camera

In [9]:
oc.homeRobot()

2025-11-25 23:33:28,827 [INFO] Homing the robot
2025-11-25 23:33:28,829 [DEBUG] Command: {"target": "robot"}
2025-11-25 23:33:28,835 [DEBUG] Starting new HTTP connection (1): 192.168.0.107:31950
2025-11-25 23:33:41,751 [DEBUG] http://192.168.0.107:31950 "POST /robot/home HTTP/1.1" 200 27
2025-11-25 23:33:41,753 [DEBUG] Response: {"message":"Homing robot."}
2025-11-25 23:33:41,754 [INFO] Robot homed successfully.


In [10]:
# Turns off MQTT broker and all controllers 
_best_effort_all_off(pumps, ultra, heat, ph, bio)

# Disconnect all controllers
beacon.stop()

pumps.disconnect()
ultra.disconnect()
heat.disconnect()
ph.disconnect()
bio.disconnect()

# Stop broker
stop_broker(proc)

logging.info(f"Experiment for all 15 wells completed and all controllers disconnected.")

2025-11-25 23:33:41,779 [INFO] [pyctl-pumps-1125_1618-d820] Published 'OFF' to pumps/01/cmd/1
2025-11-25 23:33:41,803 [INFO] [pyctl-pumps-1125_1618-d820] Published 'OFF' to pumps/01/cmd/2
2025-11-25 23:33:41,835 [INFO] [pyctl-pumps-1125_1618-d820] Published 'OFF' to pumps/01/cmd/3
2025-11-25 23:33:41,867 [INFO] [pyctl-ultra-1125_1618-343a] Published 'OFF' to ultra/01/cmd/1
2025-11-25 23:33:41,898 [INFO] [pyctl-ultra-1125_1618-343a] Published 'OFF' to ultra/01/cmd/2
2025-11-25 23:33:41,930 [INFO] [pyctl-heat-1125_1618-cf20] Published 'PWM:0' to heat/01/cmd/1
2025-11-25 23:33:41,962 [INFO] [pyctl-heat-1125_1618-cf20] Published 'OFF' to heat/01/cmd/1
2025-11-25 23:33:41,993 [INFO] [pyctl-heat-1125_1618-cf20] Published 'PWM:0' to heat/01/cmd/2
2025-11-25 23:33:42,025 [INFO] [pyctl-heat-1125_1618-cf20] Published 'OFF' to heat/01/cmd/2
2025-11-25 23:33:42,057 [INFO] [pyctl-ph-1125_1618-2d1e] Published 'STOP' to ph/01/cmd
2025-11-25 23:33:42,119 [INFO] [pyctl-bio-1125_1618-fd96] Published 'OF